In [1]:
import pandas as pd
import numpy as np
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


In [2]:
# Loading data
file_path = Path("Resources/2022-College-Guide-Main-Rankings-clear.csv")
df_rankings = pd.read_csv(file_path)
df_rankings.head()


,UnitID,Rank,Name,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,...,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank,Unnamed: 40
0,243744,1,Stanford University (CA),0.963302,6,0.949654,187,-0.021703,46,260.6667,...,0.290903,10,NO,3,0.068361,401,3,2,187,NaN
1,215062,2,University of Pennsylvania (PA),0.964294,5,0.938620,149,-0.041917,104,376.6667,...,0.160573,71,YES,6,0.246743,209,2,9,40,NaN
2,166683,3,MA Institute of Technology (MA),0.950971,10,0.990213,334,-0.032985,72,182.6667,...,0.065477,332,NO,2,0.000000,436,5,1,324,NaN
3,186131,4,Princeton University (NJ),0.976861,2,0.947812,137,-0.052740,132,153.3333,...,0.064500,339,NO,6,0.000000,436,7,6,224,NaN
4,198419,5,Duke University (NC),0.954474,9,0.967952,267,-0.014278,36,231.6667,...,0.089523,224,YES,5,0.170164,305,12,15,20,NaN


In [3]:
df_rankings.columns

Index(['UnitID', 'Rank', 'Name', '8-year graduation rate',
       'Graduation rate rank',
       'Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.',
       'Graduation rate performance rank', 'Pell/non-Pell graduation gap',
       'Pell graduation gap rank', 'Number of Pell graduates',
       'Actual vs. predicted Pell enrollment', 'Pell performance rank',
       'Median earnings 10 years after entering college',
       'Predicted median earnings 10 years after entering college',
       'Earnings performance rank',
       'Net price of attendance for families below $75,000 income',
       'Net price rank', '% of loan principal remaining 5 years later',
       'Repayment rank', 'Predicted principal remaining',
       'Repayment rate performance rank', 'Research expenditures, in millions',
       'Research expenditures rank', 'Bachelor's to PhD rank',
       'Science & engineering PhDs awarded', 'Science & engineering PhDs rank',
       'Faculty receiving sign

In [4]:
df_rankings.dtypes

UnitID                                                                            int64
Rank                                                                              int64
Name                                                                             object
8-year graduation rate                                                          float64
Graduation rate rank                                                              int64
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    float64
Graduation rate performance rank                                                  int64
Pell/non-Pell graduation gap                                                    float64
Pell graduation gap rank                                                          int64
Number of Pell graduates                                                        float64
Actual vs. predicted Pell enrollment                                            float64
Pell performance rank           

In [5]:
df_rankings.count()

UnitID                                                                          442
Rank                                                                            442
Name                                                                            442
8-year graduation rate                                                          442
Graduation rate rank                                                            442
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    442
Graduation rate performance rank                                                442
Pell/non-Pell graduation gap                                                    442
Pell graduation gap rank                                                        442
Number of Pell graduates                                                        442
Actual vs. predicted Pell enrollment                                            442
Pell performance rank                                                       

In [6]:
# Drop last column
df_rankings.drop('Unnamed: 40', axis=1, inplace=True)

In [7]:
# Encoding Carnegie column (0=NO, 1=YES)

le = LabelEncoder()
df2_rankings = df_rankings.copy()
df2_rankings['Earns Carnegie Community Engagement Classification?'] = le.fit_transform(df2_rankings['Earns Carnegie Community Engagement Classification?'])


In [8]:
df2_rankings.head()

,UnitID,Rank,Name,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,...,ROTC rank,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank
0,243744,1,Stanford University (CA),0.963302,6,0.949654,187,-0.021703,46,260.6667,...,308,0.290903,10,0,3,0.068361,401,3,2,187
1,215062,2,University of Pennsylvania (PA),0.964294,5,0.938620,149,-0.041917,104,376.6667,...,250,0.160573,71,1,6,0.246743,209,2,9,40
2,166683,3,MA Institute of Technology (MA),0.950971,10,0.990213,334,-0.032985,72,182.6667,...,12,0.065477,332,0,2,0.000000,436,5,1,324
3,186131,4,Princeton University (NJ),0.976861,2,0.947812,137,-0.052740,132,153.3333,...,34,0.064500,339,0,6,0.000000,436,7,6,224
4,198419,5,Duke University (NC),0.954474,9,0.967952,267,-0.014278,36,231.6667,...,45,0.089523,224,1,5,0.170164,305,12,15,20


In [9]:
# Define the features set.
X = df2_rankings.copy()

# Dropping target columns and unique identifier columns, etc
X = X.drop(columns=["UnitID", "Rank", "Name"])
X.head()


,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,Actual vs. predicted Pell enrollment,Pell performance rank,Median earnings 10 years after entering college,...,ROTC rank,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank
0,0.963302,6,0.949654,187,-0.021703,46,260.6667,0.038096,122,97798,...,308,0.290903,10,0,3,0.068361,401,3,2,187
1,0.964294,5,0.938620,149,-0.041917,104,376.6667,-0.000590,211,103246,...,250,0.160573,71,1,6,0.246743,209,2,9,40
2,0.950971,10,0.990213,334,-0.032985,72,182.6667,0.076769,65,111222,...,12,0.065477,332,0,2,0.000000,436,5,1,324
3,0.976861,2,0.947812,137,-0.052740,132,153.3333,0.062997,85,95689,...,34,0.064500,339,0,6,0.000000,436,7,6,224
4,0.954474,9,0.967952,267,-0.014278,36,231.6667,0.028513,142,93115,...,45,0.089523,224,1,5,0.170164,305,12,15,20


In [10]:
# Checking datatypes to make sure there are no strings
X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 37 columns):
 #   Column                                                                        Non-Null Count  Dtype  
---  ------                                                                        --------------  -----  
 0   8-year graduation rate                                                        442 non-null    float64
 1   Graduation rate rank                                                          442 non-null    int64  
 2   Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.  442 non-null    float64
 3   Graduation rate performance rank                                              442 non-null    int64  
 4   Pell/non-Pell graduation gap                                                  442 non-null    float64
 5   Pell graduation gap rank                                                      442 non-null    int64  
 6   Number of Pell graduates          

In [11]:
# Define the target set.
y = df2_rankings["Rank"].ravel()
y[:5]


array([1, 2, 3, 4, 5], dtype=int64)

In [12]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Creating a StandardScaler instance.
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [13]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (331, 37)
Training Labels Shape: (331,)
Testing Features Shape: (111, 37)
Testing Labels Shape: (111,)


## Random Forest Regressor

In [36]:
# Import the model
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf_model = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf_model=rf_model.fit(X_train_scaled, y_train)

In [37]:
# Making predictions using the testing data.
rf_predictions = rf_model.predict(X_test_scaled)


In [38]:
# Evaluate the performance with r2 score
from sklearn.metrics import r2_score
r2_rf= r2_score(y_test,rf_predictions)
print('The r2 score for this Random Forest Regressor model is:', r2_rf)

The r2 score for this Random Forest Regressor model is: 0.964116868404521


In [ ]:
# Getting accuracy data with Regressor/continuous variable model

# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))


In [ ]:
y_test

In [ ]:
errors

In [ ]:
from sklearn import metrics
# https://machinelearningmastery.com/regression-metrics-for-machine-learning

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
# Other ways of quantifying accuracy

# # Calculate mean absolute percentage error (MAPE)
# mape = 100 * (errors / y_test)
# # Calculate and display accuracy
# accuracy = 100 - np.mean(mape)
# print('Accuracy:', round(accuracy, 2), '%.')



In [ ]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances


In [ ]:
# Sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)


## Support Vector Regression

In [33]:
# Create a SVM model

# Instantiate a SVM model
from sklearn.svm import SVR

## the kernel is the most important part - try different kernels

# Fit the data

regressor = SVR(kernel = 'rbf')
svm_model = regressor.fit(X_train, y_train)
svm_model


SVR()

In [34]:
# Predicting the Test set results
svm_predictions = svm_model.predict(X_test)

In [35]:
from sklearn.metrics import r2_score
r2_svm=r2_score(y_test, svm_predictions)
print('The r2 score for this SVM model is:', r2_svm)

# r2 score with rbf kernel: The r2 score for this SVM model is: 0.10356822037319058
# r2 score with linear kernel: -209??

The r2 score for this SVM model is: 0.10356822037319058


In [ ]:
#visualize SVR results: https://www.analyticsvidhya.com/blog/2020/03/support-vector-regression-tutorial-for-machine-learning/


## Decision Tree Regression

In [21]:
# Training the Decision Tree Regression model on the Training set
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state = 0)
dt_model = dt_model.fit(X_train, y_train)
dt_model

DecisionTreeRegressor(random_state=0)

In [25]:
# Predicting the Test set results
dt_predictions = dt_model.predict(X_test)

In [26]:
# Evaluate the performance.
from sklearn.metrics import r2_score
r2_dt= r2_score(y_test,dt_predictions)
print('The r2 score for this Decision Tree model is:', r2_dt)

The r2 score for this Decision Tree model is: 0.9178296794697223


## Multiple Linear Regression

In [27]:
# Training the multiple regression model on the Training set
from sklearn.linear_model import LinearRegression
mlr_model = LinearRegression()
mlr_model = mlr_model.fit(X_train, y_train)
mlr_model

LinearRegression()

In [28]:
# Predicting the Test set results
mlr_predictions = mlr_model.predict(X_test)

In [29]:
# Evaluate the performance.
from sklearn.metrics import r2_score
r2_mlr= r2_score(y_test,mlr_predictions)
print('The r2 score for this Multiple Linear Regression model is:', r2_mlr)

The r2 score for this Multiple Linear Regression model is: 0.9763915787643959
